In [1]:
# Step 1: Import necessary libraries
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

c:\Users\Ansh Srivastava\Desktop\BERT-model\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Step 2: Load and prepare the dataset
data = pd.read_csv("updating_multiclass_dataset.csv")  # Replace with your actual file path

# Display the first few rows to check the data structure
print(data.head())\


# Map sentiments to integers (Adjust these mappings according to your dataset)
sentiment_mapping = {'positive': 2, 'negative': 0, 'neutral': 1}
data['Label'] = data['Sentiment'].map(sentiment_mapping)

# Check for NaN values in the 'Label' column
if data['Label'].isnull().any():
    print("NaN values found in labels. Please check your mapping.")
    print(data[data['Label'].isnull()])  # Print rows with NaN labels
else:
    print("All labels are mapped correctly.")

# Create a Hugging Face Dataset
dataset = Dataset.from_pandas(data)

      ID                                               Text  Label Sentiment  \
0   9536                    Cooking microwave pizzas, yummy      2  positive   
1   6135  Any plans of allowing sub tasks to show up in ...      1   neutral   
2  17697   I love the humor, I just reworded it. Like sa...      2  positive   
3  14182                       naw idk what ur talkin about      1   neutral   
4  17840          That sucks to hear. I hate days like that      0  negative   

   new_Label new_Sentiment  
0          2      Positive  
1          1       Neutral  
2          2      Positive  
3          0      Negative  
4          0      Negative  
All labels are mapped correctly.


In [3]:
# Step 3: Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
# Step 4: Define the tokenization function
def tokenize_function(examples):
    # Tokenize the text and include the labels
    tokenized_output = tokenizer(examples['Text'], padding="max_length", truncation=True)
    tokenized_output['labels'] = examples['Label']  # Add labels to the tokenized output
    return tokenized_output

In [5]:
# Step 5: Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 31232/31232 [00:13<00:00, 2272.13 examples/s]


In [6]:
# Step 6: Check if labels are correctly added
print("Sample tokenized data:", tokenized_datasets[0])

Sample tokenized data: {'ID': 9536, 'Text': 'Cooking microwave pizzas, yummy', 'Label': 2, 'Sentiment': 'positive', 'new_Label': 2, 'new_Sentiment': 'Positive', 'input_ids': [101, 8434, 18302, 10733, 2015, 1010, 9805, 18879, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [7]:
# Step 7: Split the dataset into training and validation sets
train_testvalid = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_testvalid['train']
small_train_dataset = train_dataset.shuffle(seed=42).select([i for i in list(range(1000))])  # Use 1000 samples
valid_dataset = train_testvalid['test']
small_valid_dataset = valid_dataset.shuffle(seed=42).select([i for i in list(range(200))])  # Use 200 samples

In [8]:
# Step 8: Load the BERT model for sequence classification
num_labels = len(sentiment_mapping)  # Ensure num_labels corresponds to your label mapping
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
training_args = TrainingArguments(
    output_dir="C:\\Users\\Ansh Srivastava\\Desktop\\BERT-model",
    evaluation_strategy="epoch",  # Evaluate only at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Try a higher batch size
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,  # Log every 100 steps for efficiency
    gradient_accumulation_steps=2,  # Accumulate gradients to simulate a larger batch
    fp16=True,  # Enable mixed precision training; disable if it slows down
)


c:\Users\Ansh Srivastava\Desktop\BERT-model\myenv\lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
# Step 10: Define Trainer with model, arguments, and datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset
)

In [11]:
# Step 11: Start training
try:
    trainer.train()
except Exception as e:
    print("Training failed with error:", str(e))

  0%|          | 0/1562 [00:00<?, ?it/s]c:\Users\Ansh Srivastava\Desktop\BERT-model\myenv\lib\site-packages\torch\autograd\graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\MHA.cpp:676.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
  6%|▋         | 100/1562 [01:10<16:34,  1.47it/s]

{'loss': 2.1457, 'grad_norm': 13.107839584350586, 'learning_rate': 1.8745198463508324e-05, 'epoch': 0.06}


 13%|█▎        | 200/1562 [02:18<15:31,  1.46it/s]

{'loss': 1.5625, 'grad_norm': 18.99742889404297, 'learning_rate': 1.7464788732394366e-05, 'epoch': 0.13}


 19%|█▉        | 300/1562 [03:27<14:25,  1.46it/s]

...

{'loss': 1.1144, 'grad_norm': 13.293862342834473, 'learning_rate': 2.0998719590268887e-06, 'epoch': 0.9}


 96%|█████████▌| 1500/1562 [17:54<01:02,  1.01s/it]

{'loss': 1.1438, 'grad_norm': 13.000890731811523, 'learning_rate': 8.194622279129321e-07, 'epoch': 0.96}


c:\Users\Ansh Srivastava\Desktop\BERT-model\myenv\lib\site-packages\torch\autograd\graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\MHA.cpp:676.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
                                                   
100%|██████████| 1562/1562 [21:25<00:00,  1.22it/s]

{'eval_loss': 0.5613662600517273, 'eval_runtime': 130.9924, 'eval_samples_per_second': 47.69, 'eval_steps_per_second': 5.962, 'epoch': 1.0}
{'train_runtime': 1285.2761, 'train_samples_per_second': 19.439, 'train_steps_per_second': 1.215, 'train_loss': 1.311290682354291, 'epoch': 1.0}


In [12]:
# Perform evaluation
metrics = trainer.evaluate()
print(metrics)

100%|██████████| 781/781 [02:04<00:00,  6.25it/s]

{'eval_loss': 0.5613662600517273, 'eval_runtime': 125.1365, 'eval_samples_per_second': 49.921, 'eval_steps_per_second': 6.241, 'epoch': 1.0}


In [13]:
# Define a directory to save the fine-tuned model and tokenizer
save_directory = "fine_tuned_model"  # Replace with your desired save path

# Save the model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

Model and tokenizer saved to fine_tuned_model


In [14]:
# Load the saved model and tokenizer to check if they were saved correctly
loaded_model = BertForSequenceClassification.from_pretrained(save_directory)
loaded_tokenizer = BertTokenizer.from_pretrained(save_directory)

# Print a confirmation
print("Model and tokenizer reloaded successfully from", save_directory)

Model and tokenizer reloaded successfully from fine_tuned_model
